# Topic Modeling of Historical Organic Periodicals

In [1]:
from __future__ import print_function
import glob
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

## Parameters

In [4]:
n_features = 2000 # maximum number of features (bounded by vocabulary size)
n_components = 20 # number of topics

## Load periodicals dataset

First, the periodicals dataset is loaded.

In [5]:
docs_raw = []
input_dir = "../documents_processed"
paths = glob.glob("%s/*.txt" % input_dir)
for document_filename in paths:
    with open(document_filename) as document_file:
        docs_raw.append(document_file.read())
print(len(docs_raw))

251


## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or TF-IDF form.

In [6]:
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
tf_vectorizer = CountVectorizer(stop_words = 'english',
                                max_features = n_features,
                                max_df = 0.95, 
                                min_df = 2)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(251, 2000)


In [7]:
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(251, 2000)


## Non-negative Matrix Factorization (NMF) model with Frobenius norm

In [8]:
nmf_frobenius = NMF(n_components=n_components, random_state=1,
                    alpha=.1, l1_ratio=.5)
nmf_frobenius.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(nmf_frobenius, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.321337 -0.021333       1        1  32.675961
2     -0.330145 -0.007873       2        1  20.848887
1     -0.239518  0.188651       3        1  10.705823
3     -0.255187 -0.046976       4        1   6.025429
4      0.083110 -0.188772       5        1   4.577362
5      0.096592 -0.105765       6        1   4.118158
6     -0.023051  0.285043       7        1   3.661772
13     0.128455  0.091595       8        1   2.980036
12     0.040731 -0.113694       9        1   2.032741
11     0.136552 -0.095459      10        1   2.014340
9     -0.066334 -0.215498      11        1   2.000185
10     0.051818 -0.207785      12        1   1.840356
19    -0.025497 -0.039079      13        1   1.666086
7      0.129203  0.164053      14        1   1.221405
15     0.021504  0.045235      15        1   0.831525
17     0.068877 -0.002530      16        1   0.678891
8      0.163458  0.199129      17        1   0.671725
14     0.073906  0.160761      18        1   0.603976
16     0.181811 -0.012408      19        1   0.510465
18     0.085054 -0.077296      20        1   0.334876, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
422   Default  20727.000000         corn  20727.000000  30.0000  30.0000
1623  Default  19141.000000         seed  19141.000000  29.0000  29.0000
464   Default  14270.000000        dairy  14270.000000  28.0000  28.0000
1565  Default   9934.000000         rice   9934.000000  27.0000  27.0000
307   Default   4666.000000       cheese   4666.000000  26.0000  26.0000
449   Default   9094.000000         crop   9094.000000  25.0000  25.0000
772   Default   6423.000000           gm   6423.000000  24.0000  24.0000
72    Default   4332.000000      alfalfa   4332.000000  23.0000  23.0000
755   Default   3091.000000           ge   3091.000000  22.0000  22.0000
436   Default   2865.000000        cover   2865.000000  21.0000  21.0000
1958  Default   1784.000000         whey   1784.000000  20.0000  20.0000
1688  Default   2924.000000         soil   2924.000000  19.0000  19.0000
437   Default   3057.000000          cow   3057.000000  18.0000  18.0000
1253  Default   9223.000000      organic   9223.000000  17.0000  17.0000
1132  Default   4211.000000         milk   4211.000000  16.0000  16.0000
783   Default   4087.000000        grain   4087.000000  15.0000  15.0000
690   Default   3449.000000          fda   3449.000000  14.0000  14.0000
1379  Default   2381.000000        price   2381.000000  13.0000  13.0000
1325  Default   3654.000000        plant   3654.000000  12.0000  12.0000
1915  Default   3170.000000      variety   3170.000000  11.0000  11.0000
994   Default   2897.000000     labeling   2897.000000  10.0000  10.0000
302   Default   1289.000000      chapter   1289.000000   9.0000   9.0000
259   Default   1376.000000       cancer   1376.000000   8.0000   8.0000
397   Default   2960.000000     consumer   2960.000000   7.0000   7.0000
252   Default   2322.000000   california   2322.000000   6.0000   6.0000
833   Default   1337.000000       heifer   1337.000000   5.0000   5.0000
761   Default   2596.000000  genetically   2596.000000   4.0000   4.0000
178   Default   1191.000000          bee   1191.000000   3.0000   3.0000
856   Default   2049.000000      hormone   2049.000000   2.0000   2.0000
1905  Default   1993.000000         usda   1993.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
1210  Topic20    107.079030       notice    163.316060   5.2770  -2.9207
1115  Topic20    121.025045   membership    252.491901   4.9638  -2.7983
1118  Topic20     44.096847          met    112.259015   4.7647  -3.8079
1869  Topic20    111.664446        trail    330.954209   4.6127  -2.8788
1170  Topic20     54.672433           mr    217.452698   4.3185  -3.

## NMF model with Kullback-Leibler (KL) divergence

In [9]:
nmf_kl = NMF(n_components=n_components, random_state=1,
             beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
             l1_ratio=.5)
nmf_kl.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(nmf_kl, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.010528 -0.028568       1        1  13.514042
2      0.002624 -0.012432       2        1  11.540697
10     0.099341  0.010583       3        1   9.013623
1      0.014793 -0.010826       4        1   7.481651
3     -0.118765 -0.042559       5        1   6.372635
9     -0.136083 -0.126343       6        1   6.217382
17     0.142103  0.028887       7        1   6.130157
5      0.056305 -0.097638       8        1   5.616146
15     0.078582 -0.087814       9        1   5.522742
14     0.081624 -0.034097      10        1   4.918800
16    -0.079426 -0.002660      11        1   4.208668
13     0.081176 -0.035138      12        1   4.067682
8      0.100892  0.035344      13        1   2.939814
4      0.241267  0.116112      14        1   2.909871
12     0.062335 -0.111026      15        1   2.400059
6     -0.098779  0.267643      16        1   1.876056
19    -0.127225 -0.110537      17        1   1.868045
7     -0.134694 -0.027501      18        1   1.467363
11    -0.076984  0.219317      19        1   1.107729
18    -0.178558  0.049253      20        1   0.826838, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
0     Default  2053.000000        ability  2053.000000  30.0000  30.0000
1     Default  2421.000000           able  2421.000000  29.0000  29.0000
5     Default   607.000000         accept   607.000000  28.0000  28.0000
4     Default   605.000000        academy   605.000000  27.0000  27.0000
690   Default  1070.000000            fda  1070.000000  26.0000  26.0000
699   Default  1553.000000           feed  1553.000000  25.0000  25.0000
214   Default  1283.000000            box  1283.000000  24.0000  24.0000
1936  Default  1203.000000           want  1203.000000  23.0000  23.0000
561   Default   983.000000          early   983.000000  22.0000  22.0000
618   Default  1161.000000             er  1161.000000  21.0000  21.0000
447   Default   859.000000       critical   859.000000  20.0000  20.0000
10    Default  1055.000000      according  1055.000000  19.0000  19.0000
1945  Default  1234.000000            way  1234.000000  18.0000  18.0000
1696  Default  1134.000000         source  1134.000000  17.0000  17.0000
413   Default   742.000000        control   742.000000  16.0000  16.0000
1907  Default   894.000000           used   894.000000  15.0000  15.0000
248   Default   619.000000          buyer   619.000000  14.0000  14.0000
682   Default   776.000000            far   776.000000  13.0000  13.0000
257   Default   874.000000         canada   874.000000  12.0000  12.0000
557   Default   950.000000           drug   950.000000  11.0000  11.0000
380   Default   989.000000        concern   989.000000  10.0000  10.0000
584   Default   842.000000      effective   842.000000   9.0000   9.0000
631   Default   672.000000         europe   672.000000   8.0000   8.0000
612   Default   687.000000  environmental   687.000000   7.0000   7.0000
6     Default   548.000000     acceptable   548.000000   6.0000   6.0000
244   Default   721.000000       business   721.000000   5.0000   5.0000
716   Default   818.000000          focus   818.000000   4.0000   4.0000
642   Default   713.000000       existing   713.000000   3.0000   3.0000
522   Default   973.000000       director   973.000000   2.0000   2.0000
398   Default   396.000000    consumption   396.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
571   Topic20     6.315165             ec    46.395407   2.8011  -6.6552
910   Topic20    18.647062      indicated   137.249887   2.7992  -5.5724
1524  Topic20     7.831549      reporting    57.671098   2.7987  -6.4400
1889  Topic20     6.924280     turnaround    53.195849   2.7564  -6.5631
80    Topic20    23.496631        altered   190.074397   2.7048  -5.

## Latent Dirichlet Allocation

In [10]:
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
# for TF DTM (Document-Term Matrix)
lda_tf = LatentDirichletAllocation(n_components=n_components, random_state=0, learning_method="batch")
lda_tf.fit(dtm_tf)
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.075402 -0.112138       1        1  16.164998
8      0.085649 -0.007580       2        1  13.953689
1      0.081764  0.007588       3        1  13.225134
3      0.061778 -0.006991       4        1  12.992522
9      0.102980 -0.018580       5        1  10.929454
10     0.072045  0.003419       6        1   7.918246
18     0.070197  0.018694       7        1   4.645002
4      0.051419  0.043673       8        1   3.884557
14     0.080956  0.000957       9        1   3.287191
13     0.012046  0.064534      10        1   3.256558
0      0.041550  0.045734      11        1   1.978275
2     -0.023192 -0.125747      12        1   1.767999
5      0.086053  0.012417      13        1   1.623398
7     -0.109951 -0.133213      14        1   0.990336
16     0.039175  0.100299      15        1   0.973798
11    -0.098770 -0.208856      16        1   0.944941
17    -0.394754  0.144490      17        1   0.784254
6     -0.086256 -0.013625      18        1   0.402829
15    -0.001235  0.134569      19        1   0.146899
12     0.003948  0.050356      20        1   0.129920, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
1253  Default  12907.000000      organic  12907.000000  30.0000  30.0000
1132  Default   5294.000000         milk   5294.000000  29.0000  29.0000
464   Default   4414.000000        dairy   4414.000000  28.0000  28.0000
1623  Default   5593.000000         seed   5593.000000  27.0000  27.0000
422   Default   5389.000000         corn   5389.000000  26.0000  26.0000
1379  Default   3426.000000        price   3426.000000  25.0000  25.0000
449   Default   6252.000000         crop   6252.000000  24.0000  24.0000
307   Default   1352.000000       cheese   1352.000000  23.0000  23.0000
772   Default   8493.000000           gm   8493.000000  22.0000  22.0000
437   Default   1498.000000          cow   1498.000000  21.0000  21.0000
861   Default   1180.000000        human   1180.000000  20.0000  20.0000
1536  Default   1859.000000     research   1859.000000  19.0000  19.0000
690   Default   1399.000000          fda   1399.000000  18.0000  18.0000
1759  Default   1522.000000        study   1522.000000  17.0000  17.0000
1905  Default   1881.000000         usda   1881.000000  16.0000  16.0000
823   Default   1397.000000       health   1397.000000  15.0000  15.0000
1325  Default   2644.000000        plant   2644.000000  14.0000  14.0000
804   Default    921.000000       growth    921.000000  13.0000  13.0000
72    Default    922.000000      alfalfa    922.000000  12.0000  12.0000
856   Default    914.000000      hormone    914.000000  11.0000  11.0000
1906  Default   1997.000000          use   1997.000000  10.0000  10.0000
699   Default   1943.000000         feed   1943.000000   9.0000   9.0000
64    Default   1929.000000  agriculture   1929.000000   8.0000   8.0000
1595  Default   3334.000000         said   3334.000000   7.0000   7.0000
1402  Default   1692.000000      program   1692.000000   6.0000   6.0000
1028  Default   1058.000000        level   1058.000000   5.0000   5.0000
761   Default   2641.000000  genetically   2641.000000   4.0000   4.0000
88    Default   1341.000000       animal   1341.000000   3.0000   3.0000
1522  Default   1853.000000       report   1853.000000   2.0000   2.0000
1607  Default   5576.000000          say   5576.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
859   Topic20      3.583193         http    214.614377   2.5534  -5.3712
1468  Topic20      1.813715         read    111.140246   2.5306  -6.0521
1450  Topic20      1.813715      quickly    112.312789   2.5201  -6.0521
364   Topic20      5.014124    community    318.830083   2.4936  -5.0352
43    Topic20      2.983470    advantage    192.356226   2.4797  -5.

In [11]:
# for TFIDF DTM (disabled)
# lda_tfidf = LatentDirichletAllocation(n_components=n_components, random_state=0, learning_method="batch")
# lda_tfidf.fit(dtm_tfidf)
# pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

### Using different Multi-Dimensional Scaling (MDS) functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default Principle Coordinate Analysis (PCoA) is not satisfactory.

In [12]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.071787  0.153493       1        1  16.164998
8      0.015549 -0.060511       2        1  13.953689
1     -0.006611 -0.055892       3        1  13.225134
3      0.027205 -0.019542       4        1  12.992522
9     -0.040906 -0.100462       5        1  10.929454
10     0.047952 -0.045689       6        1   7.918246
18     0.039757  0.003734       7        1   4.645002
4     -0.034935  0.025459       8        1   3.884557
14     0.099176  0.064402       9        1   3.287191
13     0.081064 -0.199762      10        1   3.256558
0      0.159066 -0.114047      11        1   1.978275
2      0.069430  0.219470      12        1   1.767999
5      0.132866 -0.022466      13        1   1.623398
7     -0.164706  0.167209      14        1   0.990336
16     0.197468  0.083398      15        1   0.973798
11    -0.065136  0.296442      16        1   0.944941
17    -0.439638 -0.103794      17        1   0.784254
6     -0.251553  0.014782      18        1   0.402829
15     0.277003 -0.050640      19        1   0.146899
12    -0.071263 -0.255584      20        1   0.129920, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
1253  Default  12907.000000      organic  12907.000000  30.0000  30.0000
1132  Default   5294.000000         milk   5294.000000  29.0000  29.0000
464   Default   4414.000000        dairy   4414.000000  28.0000  28.0000
1623  Default   5593.000000         seed   5593.000000  27.0000  27.0000
422   Default   5389.000000         corn   5389.000000  26.0000  26.0000
1379  Default   3426.000000        price   3426.000000  25.0000  25.0000
449   Default   6252.000000         crop   6252.000000  24.0000  24.0000
307   Default   1352.000000       cheese   1352.000000  23.0000  23.0000
772   Default   8493.000000           gm   8493.000000  22.0000  22.0000
437   Default   1498.000000          cow   1498.000000  21.0000  21.0000
861   Default   1180.000000        human   1180.000000  20.0000  20.0000
1536  Default   1859.000000     research   1859.000000  19.0000  19.0000
690   Default   1399.000000          fda   1399.000000  18.0000  18.0000
1759  Default   1522.000000        study   1522.000000  17.0000  17.0000
1905  Default   1881.000000         usda   1881.000000  16.0000  16.0000
823   Default   1397.000000       health   1397.000000  15.0000  15.0000
1325  Default   2644.000000        plant   2644.000000  14.0000  14.0000
804   Default    921.000000       growth    921.000000  13.0000  13.0000
72    Default    922.000000      alfalfa    922.000000  12.0000  12.0000
856   Default    914.000000      hormone    914.000000  11.0000  11.0000
1906  Default   1997.000000          use   1997.000000  10.0000  10.0000
699   Default   1943.000000         feed   1943.000000   9.0000   9.0000
64    Default   1929.000000  agriculture   1929.000000   8.0000   8.0000
1595  Default   3334.000000         said   3334.000000   7.0000   7.0000
1402  Default   1692.000000      program   1692.000000   6.0000   6.0000
1028  Default   1058.000000        level   1058.000000   5.0000   5.0000
761   Default   2641.000000  genetically   2641.000000   4.0000   4.0000
88    Default   1341.000000       animal   1341.000000   3.0000   3.0000
1522  Default   1853.000000       report   1853.000000   2.0000   2.0000
1607  Default   5576.000000          say   5576.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
859   Topic20      3.583193         http    214.614377   2.5534  -5.3712
1468  Topic20      1.813715         read    111.140246   2.5306  -6.0521
1450  Topic20      1.813715      quickly    112.312789   2.5201  -6.0521
364   Topic20      5.014124    community    318.830083   2.4936  -5.0352
43    Topic20      2.983470    advantage    192.356226   2.4797  -5.

In [13]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
19     132.200653   57.800919       1        1  16.164998
8      160.997986  162.530853       2        1  13.953689
1      -68.342567  199.951706       3        1  13.225134
3     -154.747849  101.730843       4        1  12.992522
9      169.209717 -135.533340       5        1  10.929454
10      51.799229  209.113235       6        1   7.918246
18    -138.211349   -4.155859       7        1   4.645002
4       39.489712   17.805071       8        1   3.884557
14      29.260588  -81.551201       9        1   3.287191
13      50.320316  111.262764      10        1   3.256558
0       73.850418 -176.105377      11        1   1.978275
2      114.019264  -41.887520      12        1   1.767999
5      -45.423065  102.725548      13        1   1.623398
7      236.897308   66.019432      14        1   0.990336
16    -142.851776 -144.984589      15        1   0.973798
11     216.212952  -38.664043      16        1   0.944941
17     -31.981354 -187.364975      17        1   0.784254
6      -63.173977  -82.200294      18        1   0.402829
15     -47.656647   11.757749      19        1   0.146899
12    -227.424637  -48.264793      20        1   0.129920, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
1253  Default  12907.000000      organic  12907.000000  30.0000  30.0000
1132  Default   5294.000000         milk   5294.000000  29.0000  29.0000
464   Default   4414.000000        dairy   4414.000000  28.0000  28.0000
1623  Default   5593.000000         seed   5593.000000  27.0000  27.0000
422   Default   5389.000000         corn   5389.000000  26.0000  26.0000
1379  Default   3426.000000        price   3426.000000  25.0000  25.0000
449   Default   6252.000000         crop   6252.000000  24.0000  24.0000
307   Default   1352.000000       cheese   1352.000000  23.0000  23.0000
772   Default   8493.000000           gm   8493.000000  22.0000  22.0000
437   Default   1498.000000          cow   1498.000000  21.0000  21.0000
861   Default   1180.000000        human   1180.000000  20.0000  20.0000
1536  Default   1859.000000     research   1859.000000  19.0000  19.0000
690   Default   1399.000000          fda   1399.000000  18.0000  18.0000
1759  Default   1522.000000        study   1522.000000  17.0000  17.0000
1905  Default   1881.000000         usda   1881.000000  16.0000  16.0000
823   Default   1397.000000       health   1397.000000  15.0000  15.0000
1325  Default   2644.000000        plant   2644.000000  14.0000  14.0000
804   Default    921.000000       growth    921.000000  13.0000  13.0000
72    Default    922.000000      alfalfa    922.000000  12.0000  12.0000
856   Default    914.000000      hormone    914.000000  11.0000  11.0000
1906  Default   1997.000000          use   1997.000000  10.0000  10.0000
699   Default   1943.000000         feed   1943.000000   9.0000   9.0000
64    Default   1929.000000  agriculture   1929.000000   8.0000   8.0000
1595  Default   3334.000000         said   3334.000000   7.0000   7.0000
1402  Default   1692.000000      program   1692.000000   6.0000   6.0000
1028  Default   1058.000000        level   1058.000000   5.0000   5.0000
761   Default   2641.000000  genetically   2641.000000   4.0000   4.0000
88    Default   1341.000000       animal   1341.000000   3.0000   3.0000
1522  Default   1853.000000       report   1853.000000   2.0000   2.0000
1607  Default   5576.000000          say   5576.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
859   Topic20      3.583193         http    214.614377   2.5534  -5.3712
1468  Topic20      1.813715         read    111.140246   2.5306  -6.0521
1450  Topic20      1.813715      quickly    112.312789   2.5201  -6.0521
364   Topic20      5.014124    community    318.83008